In [1]:
!pip install -Uq bitsandbytes peft trl transformers evaluate rouge-score

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.7/41.7 kB 2.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.9/72.9 MB 11.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 503.9/503.9 kB 22.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 504.6/504.6 kB 30.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.2/11.2 MB 44.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 63.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 61.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 46.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 2.2 MB/s eta 0:00:00
  

In [2]:
import torch
from huggingface_hub import login
from trl import SFTTrainer, SFTConfig
from datasets import load_dataset, DatasetDict
from peft import LoraConfig, prepare_model_for_kbit_training, get_peft_model, TaskType
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, pipeline

from google.colab import userdata
huggingface_api_key = userdata.get('huggingfaceapi')


import evaluate
import numpy as np

rouge = evaluate.load("rouge")
bleu = evaluate.load("bleu")

login(token=huggingface_api_key)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [3]:
data = load_dataset("HuggingFaceH4/ultrachat_200k")

data

README.md: 0.00B [00:00, ?B/s]

data/train_sft-00000-of-00003-a3ecf92756(…):   0%|          | 0.00/244M [00:00<?, ?B/s]

data/train_sft-00001-of-00003-0a1804bcb6(…):   0%|          | 0.00/244M [00:00<?, ?B/s]

data/train_sft-00002-of-00003-ee46ed25cf(…):   0%|          | 0.00/244M [00:00<?, ?B/s]

data/test_sft-00000-of-00001-f7dfac4afe5(…):   0%|          | 0.00/81.2M [00:00<?, ?B/s]

...

Generating train_sft split:   0%|          | 0/207865 [00:00<?, ? examples/s]

Generating test_sft split:   0%|          | 0/23110 [00:00<?, ? examples/s]

Generating train_gen split:   0%|          | 0/256032 [00:00<?, ? examples/s]

Generating test_gen split:   0%|          | 0/28304 [00:00<?, ? examples/s]

DatasetDict({
    train_sft: Dataset({
        features: ['prompt', 'prompt_id', 'messages'],
        num_rows: 207865
    })
    test_sft: Dataset({
        features: ['prompt', 'prompt_id', 'messages'],
        num_rows: 23110
    })
    train_gen: Dataset({
        features: ['prompt', 'prompt_id', 'messages'],
        num_rows: 256032
    })
    test_gen: Dataset({
        features: ['prompt', 'prompt_id', 'messages'],
        num_rows: 28304
    })
})

In [4]:
data["train_sft"][0]["messages"]

[{'content': "These instructions apply to section-based themes (Responsive 6.0+, Retina 4.0+, Parallax 3.0+ Turbo 2.0+, Mobilia 5.0+). What theme version am I using?\nOn your Collections pages & Featured Collections sections, you can easily show the secondary image of a product on hover by enabling one of the theme's built-in settings!\nYour Collection pages & Featured Collections sections will now display the secondary product image just by hovering over that product image thumbnail.\nDoes this feature apply to all sections of the theme or just specific ones as listed in the text material?",
  'role': 'user'},
 {'content': 'This feature only applies to Collection pages and Featured Collections sections of the section-based themes listed in the text material.',
  'role': 'assistant'},
 {'content': 'Can you guide me through the process of enabling the secondary image hover feature on my Collection pages and Featured Collections sections?',
  'role': 'user'},
 {'content': "Sure, here are

In [5]:
template_tokenizer = AutoTokenizer.from_pretrained("distilbert/distilgpt2")

if template_tokenizer.pad_token is None:
    template_tokenizer.pad_token = template_tokenizer.eos_token
if template_tokenizer.pad_token_id is None:
    template_tokenizer.pad_token_id = template_tokenizer.eos_token_id
if template_tokenizer.chat_template is None:
    template_tokenizer.chat_template = """<|startoftext|>
                {% for message in messages %}
                {% if message['role'] == 'system' %}System: {{ message['content'] }}\n
                {% elif message['role'] == 'user' %}User: {{ message['content'] }}\n
                {% elif message['role'] == 'assistant' %}Assistant: {{ message['content'] }} <|endoftext|>
                {% endif %}
                {% endfor %}"""

print("Pad token:", template_tokenizer.pad_token)
print("Pad token ID:", template_tokenizer.pad_token_id)
print("Chat Template:", template_tokenizer.chat_template)

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/762 [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

Pad token: <|endoftext|>
Pad token ID: 50256
Chat Template: <|startoftext|>
                {% for message in messages %}
                {% if message['role'] == 'system' %}System: {{ message['content'] }}

                {% elif message['role'] == 'user' %}User: {{ message['content'] }}

                {% elif message['role'] == 'assistant' %}Assistant: {{ message['content'] }} <|endoftext|>
                {% endif %}
                {% endfor %}


In [19]:
prep_data = DatasetDict({
    "train":data["train_sft"].shuffle(seed=2025).select(range(10)),
    "validation":data["test_sft"].shuffle(seed=2025).select(range(5)),
}).map(lambda text:{"text":template_tokenizer.apply_chat_template(text["messages"], tokenize=False)}, batched=True).remove_columns(['prompt', 'prompt_id', 'messages'])

Map:   0%|          | 0/10 [00:00<?, ? examples/s]

Map:   0%|          | 0/5 [00:00<?, ? examples/s]

In [7]:
prep_data

DatasetDict({
    train: Dataset({
        features: ['text'],
        num_rows: 100
    })
    validation: Dataset({
        features: ['text'],
        num_rows: 50
    })
})

In [8]:
print(prep_data["train"][1]["text"])

<|startoftext|>
User: Create a comprehensive marketing plan that outlines the strategies and tactics for launching a new line of eco-friendly household products. The plan should include details on the target market, product positioning, pricing, promotion, distribution channels, and sales projections. Additionally, outline the key messaging and creative concepts for the campaign, including any relevant visuals, messaging pillars, and calls-to-action. Finally, provide a timeline for the execution of the plan and a budget that outlines the resources needed to bring the products to market.

Assistant: Marketing Plan: Launching a New Line of Eco-Friendly Household Products

Executive Summary:
Our company is launching a new line of eco-friendly household products that help reduce carbon footprint and promote sustainability. The purpose of this marketing plan is to outline our strategies and tactics for successfully launching and promoting these products to our target audience. Our approach 

In [9]:
bnb_config = BitsAndBytesConfig(
                                load_in_4bit=True, # 4-bit precision model loading
                                bnb_4bit_quant_type="nf4", # Quantization type
                                bnb_4bit_compute_dtype=torch.bfloat16, # Compute type
                                bnb_4bit_use_double_quant=True, # Apply nested quantization
                                llm_int8_enable_fp32_cpu_offload=True,
                                offload_buffers=True
                               )

In [10]:
llm_model = AutoModelForCausalLM.from_pretrained(
                                                "distilbert/distilgpt2",
                                                 quantization_config=bnb_config,
                                                 device_map="auto",
                                                 torch_dtype=torch.bfloat16
                                                )

llm_model

model.safetensors:   0%|          | 0.00/353M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-5): 6 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Linear4bit(in_features=768, out_features=2304, bias=True)
          (c_proj): Linear4bit(in_features=768, out_features=768, bias=True)
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Linear4bit(in_features=768, out_features=3072, bias=True)
          (c_proj): Linear4bit(in_features=3072, out_features=768, bias=True)
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=

In [11]:
llm_pipeline = pipeline(task="text-generation", model=llm_model, tokenizer=template_tokenizer)

Device set to use cuda:0


In [12]:
llm_pipeline("Give me a short introduction to large language model.", return_full_text=False)

[{'generated_text': ' You will understand why many of the other models of language models are so complicated. I am a bit hard on your part to come up with a solution that I am not going to go through.\n\nThis is the first big step in this tutorial. I want to talk about some of the differences between languages and how to use them. In addition, I will talk about some of the advantages of working with other languages.\nFirst, I will explain the basics of the method for writing a language model. This is the first step in this tutorial.\n1. The first language model\nThe first language model is the C language model. This is the C language model. Let us go into the first language model.\nThe first language model is the C language model. This is the C language model. Let us take a look at the C language model.\nThe C language model is the C language model. Using the C language model, we can use a standard C language model.\nThe C language model is the C language model. Using the C language mo

In [13]:
prep_model = prepare_model_for_kbit_training(model=llm_model)

prep_model

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-5): 6 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Linear4bit(in_features=768, out_features=2304, bias=True)
          (c_proj): Linear4bit(in_features=768, out_features=768, bias=True)
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Linear4bit(in_features=768, out_features=3072, bias=True)
          (c_proj): Linear4bit(in_features=3072, out_features=768, bias=True)
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=

In [14]:
lora_config = LoraConfig(task_type=TaskType.CAUSAL_LM,
                         r=8, lora_alpha=16,
                         lora_dropout=0.5,
                         bias="none",
                         target_modules=['c_attn', 'c_proj', 'c_fc', 'c_proj'])

lora_config

LoraConfig(task_type=<TaskType.CAUSAL_LM: 'CAUSAL_LM'>, peft_type=<PeftType.LORA: 'LORA'>, auto_mapping=None, base_model_name_or_path=None, revision=None, inference_mode=False, r=8, target_modules={'c_fc', 'c_proj', 'c_attn'}, exclude_modules=None, lora_alpha=16, lora_dropout=0.5, fan_in_fan_out=False, bias='none', use_rslora=False, modules_to_save=None, init_lora_weights=True, layers_to_transform=None, layers_pattern=None, rank_pattern={}, alpha_pattern={}, megatron_config=None, megatron_core='megatron.core', trainable_token_indices=None, loftq_config={}, eva_config=None, corda_config=None, use_dora=False, use_qalora=False, qalora_group_size=16, layer_replication=None, runtime_config=LoraRuntimeConfig(ephemeral_gpu_offload=False), lora_bias=False, target_parameters=None)

In [15]:
quantized_model = get_peft_model(model=prep_model, peft_config=lora_config)

In [28]:
# def compute_metrics(eval_preds):
#     preds, labels = eval_preds

#     # Handle logits or tuple outputs
#     if isinstance(preds, tuple):
#         preds = preds[0]

#     # If logits are returned, get the argmax
#     if preds.ndim == 3:
#         preds = preds.argmax(-1)

#     # Convert tensors to lists if needed
#     preds = preds.tolist() if hasattr(preds, "tolist") else preds
#     labels = labels.tolist() if hasattr(labels, "tolist") else labels

#     # Simple fix: Replace -100 (ignore token) with pad token (0) and clip values
#     import numpy as np

#     # Handle labels: replace -100 with 0 and ensure valid range
#     labels = np.array(labels)
#     labels = np.where(labels == -100, 0, labels)  # Replace -100 with 0
#     labels = np.clip(labels, 0, template_tokenizer.vocab_size - 1)  # Clip to valid range
#     labels = labels.tolist()

#     # Handle predictions: ensure valid range
#     preds = np.array(preds)
#     preds = np.clip(preds, 0, template_tokenizer.vocab_size - 1)  # Clip to valid range
#     preds = preds.tolist()

#     # Decode token ids to strings
#     decoded_preds = template_tokenizer.batch_decode(preds, skip_special_tokens=True)
#     decoded_labels = template_tokenizer.batch_decode(labels, skip_special_tokens=True)

#     # Strip whitespace
#     decoded_preds = [pred.strip() for pred in decoded_preds]
#     decoded_labels = [label.strip() for label in decoded_labels]

#     # BLEU expects tokenized inputs
#     pred_tokens = [pred.split() for pred in decoded_preds]
#     label_tokens = [[label.split()] for label in decoded_labels]  # Nested list

#     # Compute metrics
#     rouge_result = rouge.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True)
#     bleu_result = bleu.compute(predictions=pred_tokens, references=label_tokens)

#     # Combine and round results
#     result = {
#         **{k: round(v * 100, 2) for k, v in rouge_result.items()},
#         "bleu": round(bleu_result["bleu"] * 100, 2)
#     }

#     return result

In [35]:
sft_config = SFTConfig(output_dir="./finetuned_qwen",
                       per_device_train_batch_size = 3,
                       per_device_eval_batch_size = 3,
                       learning_rate=2e-5,
                       num_train_epochs=10,
                       gradient_accumulation_steps=2,
                       gradient_checkpointing=True,
                       bf16=True,
                       optim="paged_adamw_32bit",
                       lr_scheduler_type="cosine",
                    #    eval_strategy="epoch",
                       report_to="none",
                       logging_strategy="epoch")

sft_trainer = SFTTrainer(model=quantized_model,
                         args=sft_config,
                         processing_class=template_tokenizer,
                         train_dataset=prep_data["train"],
                        #  eval_dataset=prep_data["validation"]
                         )

No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


In [36]:
sft_trainer.train()

Step,Training Loss
2,3.181000
4,3.180100
6,3.193900
8,3.164700
10,3.165700
12,3.159100
14,3.169600
16,3.159100
18,3.217700
20,3.195100


TrainOutput(global_step=20, training_loss=3.178612756729126, metrics={'train_runtime': 29.0297, 'train_samples_per_second': 3.445, 'train_steps_per_second': 0.689, 'total_flos': 26361154879488.0, 'train_loss': 3.178612756729126})